In [ ]:
#libraries 
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN


In [ ]:
#get dataset (subset with minhashing signatures)
data_minhashing=pd.read_csv("/Users/sararedaelli/Desktop/AMAZON REVIEWS/minhashing_results.csv")
df=data_minhashing
df.shape
df.columns

(2968263, 2)

In [ ]:
#Prepare data for clustering:
#Convert integer to float
df['signature'] = df['signature'].apply(
    lambda s: np.array(re.findall(r'-?\d+', str(s)), dtype=int)
)

#Transform  df['signature'] in lists of length 100
X = np.array(df['signature'].tolist())
y = df['rating'].values  # true labels

## CLUSTERING 

# K-MEANS 

In [ ]:
#RUN K-MEANS
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
Kmeans_clusters = kmeans.fit_predict(X)


: 

In [ ]:
#check clustering results 
Kmeans_clusters.shape
print(Kmeans_clusters[70:75])
print(np.unique(Kmeans_clusters))

## DBSCAN

In [ ]:
#DBSCAN AFTER PCA
# --- 1. Prepare data ---
X = np.array(df['signature'].tolist())

# --- 2. PCA: keep 90% cumulative variance ---
pca = PCA(n_components=0.9, random_state=42)
X_pca = pca.fit_transform(X)

print(f"Number of PCA components selected: {pca.n_components_}")
print(f"Cumulative explained variance: {np.sum(pca.explained_variance_ratio_):.2%}")

# --- 3. Estimate a good eps using the k-distance plot ---
min_samples = 3
neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors_fit = neighbors.fit(X_pca)
distances, indices = neighbors_fit.kneighbors(X_pca)

# Sort distances for the k-distance graph
distances = np.sort(distances[:, -1])

plt.figure(figsize=(8,5))
plt.plot(distances)
plt.title("k-distance graph (use the 'elbow' to choose eps)")
plt.xlabel("Points sorted by distance")
plt.ylabel(f"{min_samples}-NN distance")
plt.grid(True)
plt.show()

# 👉 Look at the plot and choose an eps value where the curve sharply increases.
# For example, eps = 0.5 (you’ll set it after visual inspection)
eps = 0.5  # <-- adjust this after checking the plot visually

# --- 4. Run DBSCAN ---
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
clusters = dbscan.fit_predict(X_pca)

In [ ]:
#DB-SCAN
# --- 1. Prepare data ---
# Assuming df['signature'] contains lists or arrays of length 100
X = np.array(df['signature'].tolist())

# --- 2. Estimate a good eps using the k-distance plot ---
min_points = 20
neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors_fit = neighbors.fit(X)
distances, indices = neighbors_fit.kneighbors(X)

# Sort distances for the k-distance graph
distances = np.sort(distances[:, -1])

plt.figure(figsize=(8,5))
plt.plot(distances)
plt.title("k-distance graph (use the 'elbow' to choose eps)")
plt.xlabel("Points sorted by distance")
plt.ylabel(f"{min_samples}-NN distance")
plt.grid(True)
plt.show()

In [ ]:
# Inspect the plot and choose an eps value around the 'elbow'
eps = 2.5 # <-- adjust after checking the curve

# --- 3. Run DBSCAN on the original data (no PCA) ---
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
Dbscan_clusters = dbscan.fit_predict(X)

## EVALUATION

In [ ]:
#EVALUATION using DAVIES-BOULDIN INDEX
from sklearn.metrics import davies_bouldin_score
# --- 3. Evaluate with Davies-Bouldin Index ---
db = davies_bouldin_score(X, clusters)
print(f"Davies-Bouldin Index (DB): {db:.4f}")

In [ ]:
#PLOT IN 2D and COLOR by clusters 
# PCA to 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Plot with legend
plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                      c=clusters, cmap='tab10', alpha=0.7)

# Add legend with cluster IDs
handles, labels = scatter.legend_elements()
plt.legend(handles, [f"Cluster {i}" for i in range(5)], title="Clusters")

plt.title("K-Means Clustering (PCA 2D)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()

In [ ]:
#PLOT IN 2D and COLOR by ratings 
# Prepare data
X = np.array(df['signature'].tolist())  # shape (n_samples, 100)
y = df['rating'].values                 # true labels

# PCA to 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Plot colored by rating
plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                      c=y, cmap='tab10', alpha=0.7)

# Add legend for unique ratings
handles, labels = scatter.legend_elements()
plt.legend(handles, [f"Rating {r}" for r in np.unique(y)], title="Ratings")

plt.title("PCA Visualization Colored by Rating")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.show()

## COMPARING CLUSTER LABELS WITH ORIGINAL RATINGS 

In [ ]:
import pandas as pd
import numpy as np

# Combine into one DataFrame
df_results = pd.DataFrame({
    'rating': y,
    'cluster': clusters
})

# Cross-tabulation to see the distribution of ratings per cluster
cluster_summary = pd.crosstab(df_results['cluster'], df_results['rating'])
print(cluster_summary)
cluster_to_rating = cluster_summary.idxmax(axis=1)
print(cluster_to_rating)



In [ ]:
#CONFUSION MATRIX 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y, predicted_rating, labels=np.unique(y))

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel("Cluster-assigned Rating")
plt.ylabel("True Rating")
plt.title("Comparison of True vs Cluster-assigned Ratings")
plt.show()

In [ ]:
#SCATTER PLOT 
plt.scatter(y, predicted_rating, alpha=0.6)
plt.xlabel("True Rating")
plt.ylabel("Cluster-assigned Rating")
plt.title("True vs Cluster-assigned Ratings")
plt.grid(True, alpha=0.3)
plt.show()
plt.figure(figsize=(8,5))
plt.scatter(y, predicted_rating, alpha=0.6)
plt.xlabel("True Rating")
plt.ylabel("Cluster-assigned Rating")
plt.title("True vs Cluster-assigned Ratings")
plt.grid(True, alpha=0.3)
plt.show()